# NN hypertuning

In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.trial import TrialState

In [9]:
train = pd.read_parquet("../data/processed/train.parquet")
train_x, train_y = train.drop("y", axis=1), train["y"]

In [10]:
test_x = pd.read_parquet("../data/processed/test.parquet")[["S", "K", "T", "r", "sigma"]]
test_y = pd.read_parquet("../data/processed/test.parquet")["y"]

In [11]:
train_x = torch.tensor(train_x.values, dtype=torch.float32)
train_y = torch.tensor(train_y.values, dtype=torch.float32).reshape(-1, 1)

test_x = torch.tensor(test_x.values, dtype=torch.float32)
test_y = torch.tensor(test_y.values, dtype=torch.float32).reshape(-1, 1)

In [12]:
dataloader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(train_x, train_y),
        batch_size=64,
        num_workers=4
    )

In [13]:
EPOCHS = 10

In [16]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 5
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.2)
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 3, 80)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = out_features
    layers.append(nn.Linear(in_features, 1))
    return nn.Sequential(*layers)


def objective(trial):
    model = define_model(trial)
    criterion = nn.MSELoss()
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for _ in range(EPOCHS):
        model.train()
        for x, y in dataloader:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred = model(test_x)
        test_loss = criterion(y_pred, test_y)

    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    
    return test_loss

In [19]:
study = optuna.create_study(study_name="hypertuning nn", direction="minimize")

study.optimize(objective, n_trials=25)

[I 2024-01-06 18:16:17,315] A new study created in memory with name: hypertuning nn
[I 2024-01-06 18:17:31,651] Trial 0 finished with value: 314.4565734863281 and parameters: {'n_layers': 1, 'dropout_rate': 0.10075036850564095, 'n_units_l0': 61, 'lr': 0.00023626438375531529}. Best is trial 0 with value: 314.4565734863281.
[I 2024-01-06 18:18:40,474] Trial 1 finished with value: 178.87303161621094 and parameters: {'n_layers': 1, 'dropout_rate': 0.1599540048628091, 'n_units_l0': 11, 'lr': 0.0029356143638398268}. Best is trial 1 with value: 178.87303161621094.
[I 2024-01-06 18:20:41,432] Trial 2 finished with value: 43.995269775390625 and parameters: {'n_layers': 2, 'dropout_rate': 0.1791285572912763, 'n_units_l0': 76, 'n_units_l1': 69, 'lr': 0.009891378437105627}. Best is trial 2 with value: 43.995269775390625.
[I 2024-01-06 18:22:12,795] Trial 3 finished with value: 212.5325164794922 and parameters: {'n_layers': 2, 'dropout_rate': 0.11685798703694757, 'n_units_l0': 7, 'n_units_l1': 10, 

In [20]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  25
  Number of pruned trials:  0
  Number of complete trials:  25
Best trial:
  Value:  6.591200828552246
  Params: 
    n_layers: 3
    dropout_rate: 0.1579730550350812
    n_units_l0: 80
    n_units_l1: 79
    n_units_l2: 79
    lr: 0.0004981149493962182
